Jaccard similarity between different configurations of oviIN modularity
* full oviINr connectome
* oviINr inputs
* oviINr outputs
* combined oviIN connectome
* oviINr connectome without oviINr
* combined oviIN connectome with oviINs

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
import os

# file path for oviIN modularity data for full ovi connectome
os.chdir('/Users/ggutierr/My Drive (ggutierr@barnard.edu)/GitHub/oviIN-analyses-gabrielle/ovi_preprocessed/preprocessed-v1.2.1')

path = os.getcwd()

# load in the data
ovi_HB_node_df = pd.read_csv('preprocessed_nodes.csv')#, index_col=0)

In [20]:
ovi_HB_node_df[0:12]

,id,key,0.0,0.05,0.1,0.5,0.75,1.0,instance,celltype,...,status,cropped,statusLabel,cellBodyFiber,somaRadius,somaLocation,roiInfo,notes,inputRois,outputRois
0,1003215282,1,1,1,1,1,1,1,CL229_R,CL229,...,Traced,False,Roughly traced,PDM19,301.0,"[23044, 14981, 11600]","{'INP': {'pre': 87, 'post': 351, 'downstream':...",NaN,"['EPA(R)', 'GOR(R)', 'IB', 'ICL(R)', 'INP', 'S...","['GOR(R)', 'IB', 'ICL(R)', 'INP', 'SCL(R)', 'S..."
1,1005952640,2,1,1,2,2,2,2,IB058_R,IB058,...,Traced,False,Roughly traced,PVL20,NaN,NaN,"{'INP': {'pre': 464, 'post': 1327, 'downstream...",NaN,"['ATL(R)', 'IB', 'ICL(R)', 'INP', 'PLP(R)', 'S...","['ATL(R)', 'IB', 'ICL(R)', 'INP', 'PLP(R)', 'S..."
2,1006928515,3,1,1,1,3,3,3,CL300_R,CL300,...,Traced,False,Roughly traced,PVL13,236.0,"[12083, 10523, 16816]","{'INP': {'pre': 79, 'post': 126, 'downstream':...",NaN,"['ATL(R)', 'IB', 'ICL(R)', 'INP', 'SCL(R)', 'S...","['ATL(R)', 'IB', 'ICL(R)', 'INP', 'SCL(R)', 'S..."
3,1007260806,4,1,2,1,4,4,4,CL301_R,CL301,...,Traced,False,Roughly traced,PVL13,236.0,"[13524, 10108, 16480]","{'INP': {'pre': 40, 'post': 128, 'downstream':...",NaN,"['GOR(R)', 'IB', 'ICL(R)', 'INP', 'PLP(R)', 'S...","['IB', 'ICL(R)', 'INP', 'PLP(R)', 'SCL(R)', 'S..."
4,1007402796,5,1,1,2,5,5,5,PS119_R,PS119,...,Traced,False,Roughly traced,PDM16,301.0,"[25364, 12010, 12544]","{'SNP(R)': {'pre': 100, 'post': 50, 'downstrea...",NaN,"['CAN(R)', 'GOR(R)', 'IB', 'ICL(L)', 'ICL(R)',...","['AVLP(R)', 'CAN(R)', 'IB', 'ICL(L)', 'INP', '..."
5,1007640554,6,2,3,3,6,6,6,IB042_R,IB042,...,Traced,False,Roughly traced,PDM24,263.0,"[16520, 19387, 6576]","{'SNP(R)': {'pre': 3, 'post': 736, 'downstream...",NaN,"['ATL(R)', 'CRE(-ROB,-RUB)(R)', 'CRE(R)', 'IB'...","['ATL(R)', 'IB', 'INP', 'SIP(R)', 'SMP(R)', 'S..."
6,1007666180,7,1,4,4,7,7,7,None,None,...,Assign,NaN,0.5assign,NaN,NaN,NaN,"{'SNP(R)': {'post': 10}, 'SMP(R)': {'post': 10}}",NaN,"['SMP(R)', 'SNP(R)']",[]
7,1007735501,8,1,1,2,8,8,8,None,None,...,Traced,True,Leaves,NaN,NaN,NaN,"{'SNP(R)': {'pre': 31, 'post': 51, 'downstream...",NaN,"['ATL(L)', 'ICL(L)', 'INP', 'SCL(L)', 'SCL(R)'...","['ATL(L)', 'ICL(L)', 'INP', 'SCL(L)', 'SCL(R)'..."
8,1008024276,9,3,5,5,9,9,9,FB5N_R,FB5N,...,Traced,False,Roughly traced,AVM08,472.5,"[19178, 29711, 37312]","{'SNP(L)': {'post': 5, 'upstream': 5, 'mito': ...",SMPCREFB5_4,"['CRE(-ROB,-RUB)(R)', 'CRE(R)', 'CX', 'FB', 'F...","['CRE(-ROB,-RUB)(R)', 'CRE(R)', 'CX', 'FB', 'F..."
9,1008085163,10,1,1,1,10,10,10,None,None,...,Traced,True,Leaves,NaN,NaN,NaN,"{'SNP(L)': {'pre': 10, 'post': 9, 'downstream'...",NaN,"['ATL(L)', 'IB', 'INP', 'SMP(L)', 'SNP(L)']","['ATL(L)', 'IB', 'INP', 'SMP(L)', 'SNP(L)']"


In [13]:
# get the coarse modules from the data
coarse_modules = ovi_HB_node_df['0.0'].unique().tolist()

In [21]:
coarse_modules

[1, 2, 3, 4, 5, 6]

In [23]:
# create a dict where the keys come from the coarse_modules list
dict_zero = {module: [] for module in coarse_modules}
dict_zero

{1: [], 2: [], 3: [], 4: [], 5: [], 6: []}

In [24]:
df = ovi_HB_node_df

for key, value in dict_zero.items():
    dict_zero[key] = df[df['0.0']==key].index.tolist()

In [26]:
chi_values = ['0.05']

In [27]:
df_test = df[df['0.0']==6]
df_test

,id,key,0.0,0.05,0.1,0.5,0.75,1.0,instance,celltype,...,status,cropped,statusLabel,cellBodyFiber,somaRadius,somaLocation,roiInfo,notes,inputRois,outputRois
45,1013891539,46,6,4,11,35,40,37,None,None,...,Traced,True,Leaves,NaN,NaN,NaN,"{'SNP(L)': {'post': 34}, 'SMP(L)': {'post': 26...",NaN,"['CRE(-RUB)(L)', 'CRE(L)', 'INP', 'SIP(L)', 'S...",[]
60,1014798169,61,6,4,4,41,47,52,None,None,...,Assign,NaN,0.5assign,NaN,NaN,NaN,"{'SNP(R)': {'post': 10}, 'SMP(R)': {'post': 10}}",NaN,"['SMP(R)', 'SNP(R)']",[]
64,1014936626,65,6,9,10,43,50,55,None,None,...,Traced,True,Leaves,NaN,NaN,NaN,"{'SNP(L)': {'pre': 12, 'post': 43, 'downstream...",NaN,"['CRE(-RUB)(L)', 'CRE(L)', 'INP', 'LAL(L)', 'L...","['CRE(-RUB)(L)', 'CRE(L)', 'INP', 'LAL(L)', 'L..."
69,1015479949,70,6,9,8,12,12,60,CRE003_a_R,CRE003_a,...,Traced,False,Roughly traced,ADL06,306.5,"[12340, 32294, 20200]","{'SNP(R)': {'pre': 20, 'post': 341, 'downstrea...",aSP14b candidates,"['ATL(R)', 'CRE(-ROB,-RUB)(R)', 'CRE(-RUB)(L)'...","['ATL(R)', 'CRE(-ROB,-RUB)(R)', 'CRE(-RUB)(L)'..."
73,1016839809,74,6,9,8,48,58,64,PAM06_a(B'2m)_R,PAM06_a,...,Traced,False,Roughly traced,ADM08,361.5,"[22659, 35722, 18416]","{'SNP(R)': {'pre': 1, 'post': 542, 'downstream...",NaN,"['CRE(-ROB,-RUB)(R)', 'CRE(-RUB)(L)', 'CRE(L)'...","['CRE(-ROB,-RUB)(R)', 'CRE(R)', 'INP', 'MB(+AC..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4532,986828383,4533,6,9,8,162,205,229,PAM13(B'1ap)_R,PAM13,...,Traced,False,Roughly traced,ADM02,292.5,"[23594, 34783, 14264]","{'SNP(R)': {'pre': 2, 'post': 84, 'downstream'...",NaN,"['CRE(-ROB,-RUB)(R)', 'CRE(-RUB)(L)', 'CRE(L)'...","['CRE(-ROB,-RUB)(R)', 'CRE(R)', 'INP', 'MB(+AC..."
4534,986974738,4535,6,9,8,622,917,1069,CRE010(ADM03)_L,CRE010,...,Traced,False,Roughly traced,NaN,292.5,"[32944, 32296, 8696]","{'SNP(L)': {'pre': 45, 'post': 83, 'downstream...",NaN,"['CRE(-RUB)(L)', 'CRE(L)', 'GC', 'INP', 'LAL(-...","['CRE(-RUB)(L)', 'CRE(L)', 'INP', 'LAL(-GA)(R)..."
4538,987273073,4539,6,9,8,573,824,1238,(PVL05)_L,None,...,Traced,False,Roughly traced,NaN,NaN,NaN,"{'SNP(R)': {'pre': 65, 'post': 52, 'downstream...",NaN,"['CRE(-ROB,-RUB)(R)', 'CRE(-RUB)(L)', 'CRE(L)'...","['CRE(-ROB,-RUB)(R)', 'CRE(-RUB)(L)', 'CRE(L)'..."
4539,987833765,4540,6,147,262,1057,1499,1783,None,None,...,Assign,NaN,0.5assign,NaN,NaN,NaN,"{'SNP(R)': {'post': 10}, 'SMP(R)': {'post': 10}}",NaN,"['SMP(R)', 'SNP(R)']",[]


In [36]:
for f, y in enumerate(chi_values):
    print(y)

0.05


In [37]:
df_test[y]

45        4
60        4
64        9
69        9
73        9
       ... 
4532      9
4534      9
4538      9
4539    147
4541     25
Name: 0.05, Length: 611, dtype: int64

In [40]:
df_test[y].unique()

array([  4,   9,  25,  29,   8,  52,  66,   2,   7,  75,   1,  84, 113,
       115, 120, 121,  24, 137, 141, 129, 147])

In [35]:
# Gets cluster numbers for resolution
cluster_all = np.zeros((len(coarse_modules),len(chi_values), 800))
for i, x in enumerate(coarse_modules):
    df_test = df[df['0.0']==x]
    for f, y in enumerate(chi_values):
        cluster = df_test[y].unique()
        cluster_all[i,f, :len(cluster)] = cluster

In [33]:
#cluster = 
df_test[chi_values]#.unique()

,0.05
45,4
60,4
64,9
69,9
73,9
...,...
4532,9
4534,9
4538,9
4539,147


In [14]:
# create a function that takes in two lists of id numbers and returns the jaccard sim of the two lists
def calculate_difference(list1, list2):
    set1 = set(list1)
    set2 = set(list2)

    unique_1 = set1-set2
    unique_2 = set2-set1

    common = set1.intersection(set2)
    total_unique = len(unique_1) + len(unique_2)
    jaccard_sim = len(common) / (total_unique + len(common))

    return unique_1, unique_2, common, jaccard_sim

# function that takes in a partition dataframe, the modules at the base resolution, and the chi value at which to compare
def get_body_ids(df, coarse_modules, chi):
    """ This function takes in a partition dataframe, the modules at the base resolution, and the chi value at which to compare. 
    It returns a dictionary of body IDs for each cluster number at the resolution and a dictionary of body IDs for each cluster 
    number at the base resolution."""
    # Need bodyIds at zero resolution for each cluster number
    dict_zero = {1: [], 2: [], 3: [], 4: [], 5: [], 6: []}
    for key, value in dict_zero.items():
        dict_zero[key] = df[df['0.0']==key].index.tolist()

    # Gets cluster numbers for resolution
    chi_values = [chi]
    cluster_all = np.zeros((len(coarse_modules),len(chi_values), 800))
    for i, x in enumerate(coarse_modules):
        df_test = df[df['0.0']==x]
        for f, y in enumerate(chi_values):
            cluster = df_test[y].unique()
            cluster_all[i,f, :len(cluster)] = cluster

    # move everything to a dictionary
    dict = {}
    for i, x in enumerate(coarse_modules):
        dict[coarse_modules[i]] = cluster_all[i,0]
    
    # Remove the zeros from the array
    for key, value in dict.items():
        dict[key] = value[value != 0]
        dict[key] = np.array(value, dtype='int64')
    
    # Get bodyIDs for each cluster number in the resolution
    dict_new = {1: [], 2: [], 3: [], 4: [], 5: [], 6: []}
    for key, value in dict.items():
        body_ids = []
        for i, x in enumerate(value):
            body_ids.extend(df[df[chi]==x].index.tolist())
        dict_new[key] = body_ids

    return dict_new, dict_zero
    
# Function that takes in partition dataframe, the modules at the base resolution, and the chi value at which to compare
def main_jaccard(df, coarse_modules, chi):
    """ This function takes in a partition dataframe, the modules at the base resolution, and the chi 
    value at which to compare. It returns a dataframe of the jaccard similarities between the resolutions at each cluster"""

    # Get the body IDs for each cluster number at the resolution and the base resolution
    dict_new, dict_zero = get_body_ids(df, coarse_modules, chi)

    # Create a dictionary to store the jaccard similarities
    jaccard_dict = {}
    for key, value in dict_new.items():
        jaccard_dict[key] = []
        unique_1, unique_2, common, jaccard_sim = calculate_difference(dict_new[key], dict_zero[key])
        jaccard_dict[key].append(jaccard_sim)
    
    # Create a dataframe to store the jaccard similarities
    df_jaccard = pd.DataFrame(jaccard_dict)
    df_jaccard.index = np.arange(1, len(df_jaccard)+1)
    
    return df_jaccard